In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-06 01:06:39--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-01-06 01:06:39 (6.39 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
#df.show()

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
#df.tail(10)

In [7]:
#df.count()

In [8]:
# Create the customers_table DataFrame
from pyspark.sql.functions import sum,avg,max,min,mean,count
customers_df = df.groupby("customer_id").agg(count("customer_id")).withColumnRenamed("count(customer_id)", "customer_count")
customers_df = customers_df.sample(fraction=0.01)
customers_df.show()

#.agg("count(customer_id)").withColumnRenamed("count(customer_id)", "customer_count")

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   24692214|             3|
|   34793766|             1|
|   43038560|             2|
|     175842|             1|
|   34939824|             2|
|   23235948|             1|
|    9015608|             1|
|   12857500|             1|
|   22548286|            11|
|     176152|             1|
|   11377791|             1|
|   41248288|             1|
|   24970677|             2|
|   32585489|             2|
|   11071690|             1|
|   38291473|             2|
|   41610655|             1|
|   31356112|             7|
|   41671434|             1|
|   45543571|             1|
+-----------+--------------+
only showing top 20 rows



In [9]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id','product_title']).drop_duplicates()
products_df = products_df.sample(fraction=0.01)
#products_df.show()

In [10]:
#products_df.filter(products_df['product_id'] == '0029148510').show()

#products_df.count()

In [11]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id','customer_id','product_id','product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = review_id_df.sample(fraction=0.01)

In [ ]:
#review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RJOVP071AVAJO|   25933450|0439873800|      84656342| 2015-08-31|
|R1ORGBETCDW3AI|    1801372|1623953553|     729938122| 2015-08-31|
| R7TNRFQAOUTX5|    5782091|142151981X|     678139048| 2015-08-31|
|R2GANXKDIFZ6OI|   32715830|014241543X|     712432151| 2015-08-31|
|R2NYB6C3R8LVN6|   14005703|1604600527|     800572372| 2015-08-31|
|R13U5PBJI1H94K|   36205738|0399170863|     559876774| 2015-08-31|
|R1H8UVH990F8VE|   44121167|1517007240|     299984591| 2015-08-31|
|R2MC0N30WZMRQ5|   16519255|0671728725|     821650353| 2015-08-31|
| R2NO2HXK16Y4J|   49361350|1111349533|     419457767| 2015-08-31|
|R245YIAVJK82ZL|   11423253|812211637X|     285887177| 2015-08-31|
| RFBMLBWQOZ1UM|   37536260|0679413359|      58983679| 2015-08-31|
| RHE1FEWV3KTQH|   23614950|0983990662|     222024699| 2015-08

In [12]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase' ])
vine_df = vine_df.sample(fraction=0.01)
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2G820E5QUOS58|          5|            0|          2|   N|                Y|
|R18DJX9FOO2CES|          5|            0|          0|   N|                Y|
|R3L5MFHHTE91JG|          5|            2|          3|   N|                Y|
|R3DSAWB42S2B5R|          5|            0|          0|   N|                Y|
| RZPOFOGI76N52|          5|            0|          0|   N|                Y|
|R1JJR154OSVA4G|          5|            2|          2|   N|                Y|
|R1GNVDSA08AX0G|          5|            0|          0|   N|                Y|
|R3LMTPWYUKWY3T|          5|            0|          0|   N|                N|
|R1QSS59SU97RHY|          5|            0|          0|   N|                N|
|R13HJTGY838NFM|          1|            0|          6|   N|     

In [13]:
vine_df.filter(vine_df['vine'] == 'Y').show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQOEQZH3XHU1|          5|            2|          3|   Y|                N|
|R1F86CLSECRANC|          5|            0|          1|   Y|                N|
|R3IZJXG17R2FJ5|          5|            0|          0|   Y|                N|
|R1T45ZVHQRUDTQ|          5|            1|          1|   Y|                N|
|R3SMJJVD4F8SUL|          3|            0|          0|   Y|                N|
|R1QQ0ZZPS0VMQM|          2|            0|          0|   Y|                N|
| RO0UMRWVNPTZQ|          3|            0|          0|   Y|                N|
|R2NWBRK68RPZ98|          4|            0|          0|   Y|                N|
| RUEJEIFY7441H|          4|            0|          0|   Y|                N|
|R11C695AJ3S4EX|          5|            0|          0|   Y|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [14]:
# store environmental variable
from getpass import getpass
password = getpass('Enter Database Password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://challenge-db.czzemirikcit.us-east-2.rds.amazonaws.com:5432/challenge-db"
config = {"user":"postgres", 
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter Database Password··········


In [15]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [16]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [17]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [18]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)